In [1]:
# Installing Libraries
# !pip install tflearn

In [2]:
# pip install tflearn

In [3]:
#Imports
import nltk
#nltk.download('punkt')
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
#Loading the intents data
with open("intents.json") as file:
	data = json.load(file)

In [5]:
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

In [6]:
#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    #Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])

In [7]:
#Pre-Processing the given data
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [8]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

In [9]:
#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)

In [10]:
#Generating and Saving ML Model
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 1199  | total loss: 0.11812 | time: 0.027s
| Adam | epoch: 200 | loss: 0.11812 - acc: 0.9999 -- iter: 40/43
Training Step: 1200  | total loss: 0.86831 | time: 0.032s
| Adam | epoch: 200 | loss: 0.86831 - acc: 0.9124 -- iter: 43/43
--
INFO:tensorflow:d:\Alwi\TUGAS KULIAH SEMESTER 8\chatbot_yaelahwi\model\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
